In [1]:
import io
import subprocess
import os
import glob
import sys
import pwd

import pandas as pd
import numpy as np
import scipy
import scipy.stats as stats
from sklearn.decomposition import PCA
import pickle
import re
import math
import plotnine as pn
import seaborn as sns
import glob
import subprocess
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from numpy import log
from collections import defaultdict,Counter
import re
import statsmodels
import warnings
import statsmodels.stats.multitest as smm
import json

from pathlib import Path
#from midas import m
#from datetime import datetime, date
#import altair as alt

# gh specific
sys.path.append('/ghsfa/projects/pharma/shared_scripts/')
import bibp.functions as functions
import hgvslib.pHGVS as pHGVS  ## do a pull on the hgvslib git repo

# !pip install statannotations==0.4.0
# #!pip uninstall statannotations --yes
# from statannotations.Annotator import Annotator

warnings.filterwarnings("ignore")
pd.options.display.max_rows = None
pd.options.display.max_columns=None
pd.options.display.max_colwidth=None
work_dir="/ghsfa/projects/pharma/projects/sirius_pharma/hazhang_projects/AZ_TFv2_update_SOW172_01082025"

# SOW172 TFv2 update (different specificity cutoffs 98%, 97%, 95% and 90%)

## Data preocessing

In [2]:
# Readin latest batches (TFv2 updated version by Evan) from Google drive

# Infinity batch
# df_sow172_all = pd.read_csv("/ghsfa/projects/pharma/datawarehouse/Infinity/20241114_AstraZeneca_SOW202_Batch1_80Samples_InfinityReport.csv", header = 1)
df_sow172_all = pd.read_csv(f"{work_dir}/20240620_AstraZeneca_SOW172_Batch1_340Samples_TFv2_InfinityReport.csv", header = 0) 


print("Total sample size of SOW172 Infinity cumulative batches is " + str(df_sow172_all.shape[0]) + "\n") 
print("Total unique sample size of SOW172 Infinity cumulative batches is " + str(df_sow172_all.GHSampleID.nunique()) + "\n")
print("Total unique patient sample size of SOW172 Infinity cumulative batches is " + str(df_sow172_all.Patient_ID.nunique()) + "\n")

df_sow172_all.head()

Total sample size of SOW172 Infinity cumulative batches is 11731

Total unique sample size of SOW172 Infinity cumulative batches is 340

Total unique patient sample size of SOW172 Infinity cumulative batches is 281



,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Visit_name,Alt-NoAlt,Sample_status,Sample_comment,Variant_type,Indel_type,Gene,Chromosome,Position,Exon,Mut_aa,Mut_nt,Mut_cdna,Transcript,Percentage,Splice_effect,Somatic_status,Molecular_consequence,Fusion_chrom_b,Fusion_gene_b,Fusion_position_a,Fusion_position_b,Direction_a,Direction_b,Downstream_gene,Copy_number,CNV_type,COSMIC,dbSNP,ClinVar,ClinVarID,Functional_impact,Mutant_allele_status,Mol_count,Max_percentage,Alleletype,HRD_score,Tumor_methylation_status,Tumor_methylation_score,TMB_score,TMB_category,MSI_High,cfDNA_ng,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,Cancertype,Practice_name,Physician_name
0,N4496000000,130869837_0,A0931211,B00578713,130869837,0,1,SUCCESS,NaN,SNV,NaN,NOVA1,14,27066580.0,1.0,D21D,G>A,c.63C>T,NM_002515.3,52.41,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs145050329,NaN,NaN,NaN,NaN,806.0,1.71,NaN,NaN,Detected,0.000671,10.63,Low,Not Detected,26.1,3.5,NaN,12/14/23,12/14/22,3/15/24,CRC,Not Applicable,Not Applicable
1,N4496000000,130869837_0,A0931211,B00578713,130869837,0,1,SUCCESS,NaN,SNV,NaN,JAK1,1,65348981.0,3.0,I62V,T>C,c.184A>G,NM_002227.4,51.34,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV99071479,rs202021264,not_provided,138287.0,NaN,NaN,1291.0,1.71,NaN,NaN,Detected,0.000671,10.63,Low,Not Detected,26.1,3.5,NaN,12/14/23,12/14/22,3/15/24,CRC,Not Applicable,Not Applicable
2,N4496000000,130869837_0,A0931211,B00578713,130869837,0,1,SUCCESS,NaN,SNV,NaN,EML4,2,42510096.0,8.0,T309P,A>C,c.925A>C,NM_019063.5,51.10,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1722.0,1.71,NaN,NaN,Detected,0.000671,10.63,Low,Not Detected,26.1,3.5,NaN,12/14/23,12/14/22,3/15/24,CRC,Not Applicable,Not Applicable
3,N4496000000,130869837_0,A0931211,B00578713,130869837,0,1,SUCCESS,NaN,SNV,NaN,MRE11,11,94204784.0,8.0,G267G,T>C,c.801A>G,NM_005590.4,50.65,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs769856404,NaN,NaN,NaN,NaN,1448.0,1.71,NaN,NaN,Detected,0.000671,10.63,Low,Not Detected,26.1,3.5,NaN,12/14/23,12/14/22,3/15/24,CRC,Not Applicable,Not Applicable
4,N4496000000,130869837_0,A0931211,B00578713,130869837,0,1,SUCCESS,NaN,SNV,NaN,VEGFB,11,64005105.0,6.0,*208Yext*?,G>C,c.624G>C,NM_003377.5,50.59,NaN,germline,stop_lost,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs1033683825,NaN,NaN,NaN,NaN,1188.0,1.71,NaN,NaN,Detected,0.000671,10.63,Low,Not Detected,26.1,3.5,NaN,12/14/23,12/14/22,3/15/24,CRC,Not Applicable,Not Applicable


In [3]:
# Remove duplication
df_sow172_dedup = df_sow172_all.drop_duplicates(subset = "GHSampleID")
df_sow172_dedup.head()

,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Visit_name,Alt-NoAlt,Sample_status,Sample_comment,Variant_type,Indel_type,Gene,Chromosome,Position,Exon,Mut_aa,Mut_nt,Mut_cdna,Transcript,Percentage,Splice_effect,Somatic_status,Molecular_consequence,Fusion_chrom_b,Fusion_gene_b,Fusion_position_a,Fusion_position_b,Direction_a,Direction_b,Downstream_gene,Copy_number,CNV_type,COSMIC,dbSNP,ClinVar,ClinVarID,Functional_impact,Mutant_allele_status,Mol_count,Max_percentage,Alleletype,HRD_score,Tumor_methylation_status,Tumor_methylation_score,TMB_score,TMB_category,MSI_High,cfDNA_ng,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,Cancertype,Practice_name,Physician_name
0,N4496000000,130869837_0,A0931211,B00578713,130869837,0,1,SUCCESS,NaN,SNV,NaN,NOVA1,14,27066580.0,1.0,D21D,G>A,c.63C>T,NM_002515.3,52.41,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs145050329,NaN,NaN,NaN,NaN,806.0,1.71,NaN,NaN,Detected,0.000671,10.63,Low,Not Detected,26.10,3.5,NaN,12/14/23,12/14/22,3/15/24,CRC,Not Applicable,Not Applicable
27,N4496000000,151684_0,A0931212,B00578712,151684,0,0,FAIL,Possible Contamination,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.53,2.5,NaN,12/26/23,1/1/22,3/15/24,CRC,Not Applicable,Not Applicable
28,N4496000000,17-102402_1,A0931213,B00578814,17-102402,1,1,SUCCESS,NaN,SNV,NaN,PTPRT,20,41420014.0,3.0,I103L,T>G,c.307A>C,NM_133170.4,53.29,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs200831045,NaN,NaN,NaN,NaN,3235.0,2.39,NaN,NaN,Detected,0.056752,12.15,Low,Not Detected,20.02,3.5,NaN,12/14/23,4/4/23,3/15/24,LUNG,Not Applicable,Not Applicable
67,N4496000000,1044-4_0,A0931214,B00578812,1044-4,0,1,SUCCESS,NaN,SNV,NaN,PIM1,6,37138127.0,1.0,P17S,C>T,c.49C>T,NM_001243186.1,53.69,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs962945709,NaN,NaN,NaN,NaN,599.0,11.50,NaN,NaN,Not Detected,0.000000,6.62,Low,Not Detected,16.59,4.0,NaN,12/19/23,NaN,3/15/24,LUNG,Not Applicable,Not Applicable
106,N4496000000,24002_1,A0931215,B00578817,24002,1,1,SUCCESS,NaN,SNV,NaN,FAT1,4,187540610.0,10.0,T2377M,G>A,c.7130C>T,NM_005245.4,54.12,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV99081608,rs201363601,NaN,NaN,NaN,NaN,2197.0,2.24,NaN,NaN,Detected,0.060444,NaN,Not evaluable,Not Detected,18.30,4.0,NaN,12/14/23,1/17/23,3/15/24,LUNG,Not Applicable,Not Applicable


In [4]:
# Pull "runid" from ghdb
df_sow172_1 = df_sow172_dedup.copy()

# Define the IDs you want to query for
ids = tuple(df_sow172_1['GHSampleID'].unique())

# Generate the SQL query string
query = "SELECT * FROM gh_sample WHERE run_sample_id IN {}".format(ids)

# Get the dataframe with "runid"
sample_df = functions.run_sql_query(query)
sample_df = sample_df.rename(columns={"run_sample_id": "GHSampleID"})

# Found some sample has duplicated runid which might due to re-run. So only the latest FC runs were saved.
# Extract the date from runid and convert to datetime format for accurate comparison
sample_df['run_date'] = pd.to_datetime(sample_df['runid'].str.extract(r'(\d{6})')[0], format='%y%m%d')
df_sow172_1 = df_sow172_1.merge(sample_df[['GHSampleID', 'runid', 'run_date']], on='GHSampleID', how="left")
df_sow172_1 = df_sow172_1.sort_values(by=['GHSampleID', 'run_date'], ascending=[True, False]).drop_duplicates(subset='GHSampleID', keep='first')
df_sow172_1 = df_sow172_1.drop(columns='run_date')

print("Total sample size of SOW172 Infinity cumulative batches after merging runid is " + str(df_sow172_1.shape[0])) 
df_sow172_1.head()


Total sample size of SOW172 Infinity cumulative batches after merging runid is 340


,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Visit_name,Alt-NoAlt,Sample_status,Sample_comment,Variant_type,Indel_type,Gene,Chromosome,Position,Exon,Mut_aa,Mut_nt,Mut_cdna,Transcript,Percentage,Splice_effect,Somatic_status,Molecular_consequence,Fusion_chrom_b,Fusion_gene_b,Fusion_position_a,Fusion_position_b,Direction_a,Direction_b,Downstream_gene,Copy_number,CNV_type,COSMIC,dbSNP,ClinVar,ClinVarID,Functional_impact,Mutant_allele_status,Mol_count,Max_percentage,Alleletype,HRD_score,Tumor_methylation_status,Tumor_methylation_score,TMB_score,TMB_category,MSI_High,cfDNA_ng,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,Cancertype,Practice_name,Physician_name,runid
3,N4496000000,151684_0,A0931212,B00578712,151684,0,0,FAIL,Possible Contamination,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.53,2.5,NaN,12/26/23,1/1/22,3/15/24,CRC,Not Applicable,Not Applicable,240206_A02048_0117_AHNNYCDSX7
1,N4496000000,130869837_0,A0931211,B00578713,130869837,0,1,SUCCESS,NaN,SNV,NaN,NOVA1,14,27066580.0,1.0,D21D,G>A,c.63C>T,NM_002515.3,52.41,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs145050329,NaN,NaN,NaN,NaN,806.0,1.71,NaN,NaN,Detected,0.000671,10.63,Low,Not Detected,26.10,3.5,NaN,12/14/23,12/14/22,3/15/24,CRC,Not Applicable,Not Applicable,240206_A02048_0117_AHNNYCDSX7
52,N4496000000,23986_1,A0931261,B00578731,23986,1,1,SUCCESS,NaN,SNV,NaN,FZD7,2,202899471.0,1.0,P34L,C>T,c.101C>T,NM_003507.2,51.68,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs772362388,NaN,NaN,NaN,NaN,1292.0,0.39,NaN,NaN,Not Detected,0.000000,NaN,Not evaluable,Not Detected,183.04,4.0,NaN,12/14/23,1/26/23,3/15/24,LUNG,Not Applicable,Not Applicable,240111_A02019_0030_AHMHTTDSX7
51,N4496000000,200058744_0,A0931260,B00578732,200058744,0,1,SUCCESS,NaN,SNV,NaN,GLI1,12,57865247.0,12.0,L908L,A>G,c.2724A>G,NM_005269.3,51.04,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3879.0,2.77,NaN,NaN,Not Detected,0.000000,9.45,Low,Not Detected,87.36,3.5,NaN,12/14/23,1/7/21,3/15/24,CRC,Not Applicable,Not Applicable,240111_A02019_0030_AHMHTTDSX7
54,N4496000000,214565_0,A0931263,B00578733,214565,0,1,SUCCESS,NaN,SNV,NaN,RECQL4,8,145739846.0,10.0,R562W,G>A,c.1684C>T,NM_004260.4,54.23,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV52881129,rs61759892,NaN,NaN,NaN,NaN,2560.0,4.71,NaN,NaN,Detected,0.035111,11.35,Low,Not Detected,18.72,2.0,NaN,12/26/23,1/1/22,3/15/24,CRC,Not Applicable,Not Applicable,240111_A02019_0030_AHMHTTDSX7


In [5]:
def add_fc_dir_column(df):
    """
    Adds a 'fc_dir' column to the given dataframe, specifying the FC path based on the 'runid' column.

    Args:
        df (pd.DataFrame): A DataFrame that must contain a 'runid' column. This column is used to
                           match directory paths from predefined locations.

    Returns:
        pd.DataFrame: The original dataframe with an added 'fc_dir' column indicating the directory path
                      corresponding to each 'runid'. If no directory is found, the path will be an empty string.
        pd.DataFrame: A DataFrame containing the rows for which no directory path was found.
    """
    # Define the path
    fc_dir_all = "/ghsfa/ivd/flowcentral"
    fc_dir_omni = "/ghsfa/projects/omni_v2/flowcells"
    
    # Initialize a list to store rows where no BIP output was found
    rows_no_found = []
    
    # Initialize the new column in the DataFrame
    df['fc_dir'] = ''
    
    # Iterate over the DataFrame rows
    for index, row in df.iterrows():
        runid = row["runid"]
        
        # Find original folder
        pattern_ori = f"{fc_dir_all}/*{runid}*"
        folders = glob.glob(pattern_ori)
        if folders:
            for folder in folders:
                primary_fc_dir = folder
                # Check if there is a 'sirius' folder under the primary fc_dir. This is for Sirius 1.5 folder structure change.
                if os.path.exists(os.path.join(primary_fc_dir, "sirius")):
                    primary_fc_dir = os.path.join(primary_fc_dir, "sirius")
                df.loc[index, 'fc_dir'] = primary_fc_dir
        else:
            # Search in the alternative directory if no folder found in the first directory
            pattern_omni = f"{fc_dir_omni}/*{runid}*"
            folders_omni = glob.glob(pattern_omni)
            if folders_omni:
                df.loc[index, 'fc_dir'] = folders_omni[0]
            else:
                # Append the row to the list if no folder is found in either location
                rows_no_found.append(row)
    
    # Create a DataFrame with rows where folders were not found in any sources
    df_no_fc_dir = pd.DataFrame(rows_no_found)
    
    return df, df_no_fc_dir

# Generate the new dataframe 
df_sow172_2, df_no_found = add_fc_dir_column(df_sow172_1)

print("Total sample size of SOW172 Infinity cumulative batches with fc_dir is " + str(df_sow172_2.shape[0]) + "\n") 
print("Total sample size of SOW172 Infinity cumulative batches with NO fc_dir is " + str(df_no_found.shape[0]) + "\n") 

df_sow172_2.head()



Total sample size of SOW172 Infinity cumulative batches with fc_dir is 340

Total sample size of SOW172 Infinity cumulative batches with NO fc_dir is 0



,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Visit_name,Alt-NoAlt,Sample_status,Sample_comment,Variant_type,Indel_type,Gene,Chromosome,Position,Exon,Mut_aa,Mut_nt,Mut_cdna,Transcript,Percentage,Splice_effect,Somatic_status,Molecular_consequence,Fusion_chrom_b,Fusion_gene_b,Fusion_position_a,Fusion_position_b,Direction_a,Direction_b,Downstream_gene,Copy_number,CNV_type,COSMIC,dbSNP,ClinVar,ClinVarID,Functional_impact,Mutant_allele_status,Mol_count,Max_percentage,Alleletype,HRD_score,Tumor_methylation_status,Tumor_methylation_score,TMB_score,TMB_category,MSI_High,cfDNA_ng,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,Cancertype,Practice_name,Physician_name,runid,fc_dir
3,N4496000000,151684_0,A0931212,B00578712,151684,0,0,FAIL,Possible Contamination,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.53,2.5,NaN,12/26/23,1/1/22,3/15/24,CRC,Not Applicable,Not Applicable,240206_A02048_0117_AHNNYCDSX7,/ghsfa/projects/omni_v2/flowcells/240206_A02048_0117_AHNNYCDSX7_PS3C358LDT_AP3.Sirius-1.5.0-RC4
1,N4496000000,130869837_0,A0931211,B00578713,130869837,0,1,SUCCESS,NaN,SNV,NaN,NOVA1,14,27066580.0,1.0,D21D,G>A,c.63C>T,NM_002515.3,52.41,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs145050329,NaN,NaN,NaN,NaN,806.0,1.71,NaN,NaN,Detected,0.000671,10.63,Low,Not Detected,26.10,3.5,NaN,12/14/23,12/14/22,3/15/24,CRC,Not Applicable,Not Applicable,240206_A02048_0117_AHNNYCDSX7,/ghsfa/projects/omni_v2/flowcells/240206_A02048_0117_AHNNYCDSX7_PS3C358LDT_AP3.Sirius-1.5.0-RC4
52,N4496000000,23986_1,A0931261,B00578731,23986,1,1,SUCCESS,NaN,SNV,NaN,FZD7,2,202899471.0,1.0,P34L,C>T,c.101C>T,NM_003507.2,51.68,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs772362388,NaN,NaN,NaN,NaN,1292.0,0.39,NaN,NaN,Not Detected,0.000000,NaN,Not evaluable,Not Detected,183.04,4.0,NaN,12/14/23,1/26/23,3/15/24,LUNG,Not Applicable,Not Applicable,240111_A02019_0030_AHMHTTDSX7,/ghsfa/projects/omni_v2/flowcells/240111_A02019_0030_AHMHTTDSX7_PS3C359LDT_AP2.Sirius-1.1.2-RLS
51,N4496000000,200058744_0,A0931260,B00578732,200058744,0,1,SUCCESS,NaN,SNV,NaN,GLI1,12,57865247.0,12.0,L908L,A>G,c.2724A>G,NM_005269.3,51.04,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3879.0,2.77,NaN,NaN,Not Detected,0.000000,9.45,Low,Not Detected,87.36,3.5,NaN,12/14/23,1/7/21,3/15/24,CRC,Not Applicable,Not Applicable,240111_A02019_0030_AHMHTTDSX7,/ghsfa/projects/omni_v2/flowcells/240111_A02019_0030_AHMHTTDSX7_PS3C359LDT_AP2.Sirius-1.1.2-RLS
54,N4496000000,214565_0,A0931263,B00578733,214565,0,1,SUCCESS,NaN,SNV,NaN,RECQL4,8,145739846.0,10.0,R562W,G>A,c.1684C>T,NM_004260.4,54.23,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV52881129,rs61759892,NaN,NaN,NaN,NaN,2560.0,4.71,NaN,NaN,Detected,0.035111,11.35,Low,Not Detected,18.72,2.0,NaN,12/26/23,1/1/22,3/15/24,CRC,Not Applicable,Not Applicable,240111_A02019_0030_AHMHTTDSX7,/ghsfa/projects/omni_v2/flowcells/240111_A02019_0030_AHMHTTDSX7_PS3C359LDT_AP2.Sirius-1.1.2-RLS


In [6]:
# Pull the archived FC by Starfish

# Mapping dictionary for correct fc_dir values
fc_dir_mapping = {
    "240111_A02019_0030_AHMHTTDSX7": "/ghsfa/projects/bioinformatics/starfish/flowcentral240111_A02019_0030_AHMHTTDSX7.a6632f4d-dc3f-44f0-9ae1-0a8b55716eb1.20240113072253",
    "240111_A01552_0369_BHMHHLDSX7": "/ghsfa/projects/bioinformatics/starfish/flowcentral240111_A01552_0369_BHMHHLDSX7.a37698ce-13f4-45b6-947e-4fb691daf9f4.20240113082257",
    "240111_A02019_0031_BHMJHHDSX7": "/ghsfa/projects/bioinformatics/starfish/flowcentral240111_A02019_0031_BHMJHHDSX7.1ab300c1-884a-4b36-997c-bafcd5009831.20240113072621",
    "240112_A01613_0302_BHMMTFDSX7": "/ghsfa/projects/bioinformatics/starfish/flowcentral240112_A01613_0302_BHMMTFDSX7.cea2fa14-4b1d-4b37-844d-6525df7b3026.20240113235712",
    "240112_A01613_0303_AHMJWMDSX7": "/ghsfa/projects/bioinformatics/starfish/flowcentral240112_A01613_0303_AHMJWMDSX7.0013d865-965c-465f-9241-996e93e32e70.20240114000044",
    "240112_A01552_0370_AHMHTVDSX7": "/ghsfa/projects/bioinformatics/starfish/flowcentral240112_A01552_0370_AHMHTVDSX7.d062574e-4bff-4ef2-afe1-05a47aac2565.20240114011348",
    "240113_A01810_0294_AHMMTGDSX7": "/ghsfa/projects/bioinformatics/starfish/flowcentral240113_A01810_0294_AHMMTGDSX7.838c525c-3c6c-427e-a21b-e75de9c10a9e.20240115115421",
    "240113_A01810_0295_BHMJK7DSX7": "/ghsfa/projects/bioinformatics/starfish/flowcentral240113_A01810_0295_BHMJK7DSX7.de53cf23-14a0-4f78-8858-42a6051599f3.20240115115751",
    "240113_A01021_0602_AHMMTJDSX7": "/ghsfa/projects/bioinformatics/starfish/flowcentral240113_A01021_0602_AHMMTJDSX7.0d32760f-c089-45b9-9890-a9400d19994d.20240115122427",
    "240114_A02019_0033_BHMHNYDSX7": "/ghsfa/projects/bioinformatics/starfish/flowcentral240114_A02019_0033_BHMHNYDSX7.f4f1c92b-92f4-4115-9b59-353bc9967c78.20240116000137",
    "240114_A00441_0929_AHMJ3JDSX7": "/ghsfa/projects/bioinformatics/starfish/flowcentral240114_A00441_0929_AHMJ3JDSX7.a1624781-ce11-4a13-a63e-dd663b1241dd.20240116045201",
    "240127_A01361_0594_BHMKVWDSX7": "/ghsfa/projects/bioinformatics/starfish/flowcentral240127_A01361_0594_BHMKVWDSX7.50e114bd-b752-410b-8a03-18d3fb446e1d.20240129181640",
    "240206_A02048_0117_AHNNYCDSX7": "/ghsfa/projects/bioinformatics/starfish/flowcentral240206_A02048_0117_AHNNYCDSX7.1da433ba-2782-431f-a56a-b7257055cb67.20240207222712"
}

# Replacing incorrect "fc_dir" values using the mapping
df_sow172_2["fc_dir"] = df_sow172_2["runid"].map(fc_dir_mapping)

df_sow172_2.head()


,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Visit_name,Alt-NoAlt,Sample_status,Sample_comment,Variant_type,Indel_type,Gene,Chromosome,Position,Exon,Mut_aa,Mut_nt,Mut_cdna,Transcript,Percentage,Splice_effect,Somatic_status,Molecular_consequence,Fusion_chrom_b,Fusion_gene_b,Fusion_position_a,Fusion_position_b,Direction_a,Direction_b,Downstream_gene,Copy_number,CNV_type,COSMIC,dbSNP,ClinVar,ClinVarID,Functional_impact,Mutant_allele_status,Mol_count,Max_percentage,Alleletype,HRD_score,Tumor_methylation_status,Tumor_methylation_score,TMB_score,TMB_category,MSI_High,cfDNA_ng,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,Cancertype,Practice_name,Physician_name,runid,fc_dir
3,N4496000000,151684_0,A0931212,B00578712,151684,0,0,FAIL,Possible Contamination,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.53,2.5,NaN,12/26/23,1/1/22,3/15/24,CRC,Not Applicable,Not Applicable,240206_A02048_0117_AHNNYCDSX7,/ghsfa/projects/bioinformatics/starfish/flowcentral240206_A02048_0117_AHNNYCDSX7.1da433ba-2782-431f-a56a-b7257055cb67.20240207222712
1,N4496000000,130869837_0,A0931211,B00578713,130869837,0,1,SUCCESS,NaN,SNV,NaN,NOVA1,14,27066580.0,1.0,D21D,G>A,c.63C>T,NM_002515.3,52.41,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs145050329,NaN,NaN,NaN,NaN,806.0,1.71,NaN,NaN,Detected,0.000671,10.63,Low,Not Detected,26.10,3.5,NaN,12/14/23,12/14/22,3/15/24,CRC,Not Applicable,Not Applicable,240206_A02048_0117_AHNNYCDSX7,/ghsfa/projects/bioinformatics/starfish/flowcentral240206_A02048_0117_AHNNYCDSX7.1da433ba-2782-431f-a56a-b7257055cb67.20240207222712
52,N4496000000,23986_1,A0931261,B00578731,23986,1,1,SUCCESS,NaN,SNV,NaN,FZD7,2,202899471.0,1.0,P34L,C>T,c.101C>T,NM_003507.2,51.68,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs772362388,NaN,NaN,NaN,NaN,1292.0,0.39,NaN,NaN,Not Detected,0.000000,NaN,Not evaluable,Not Detected,183.04,4.0,NaN,12/14/23,1/26/23,3/15/24,LUNG,Not Applicable,Not Applicable,240111_A02019_0030_AHMHTTDSX7,/ghsfa/projects/bioinformatics/starfish/flowcentral240111_A02019_0030_AHMHTTDSX7.a6632f4d-dc3f-44f0-9ae1-0a8b55716eb1.20240113072253
51,N4496000000,200058744_0,A0931260,B00578732,200058744,0,1,SUCCESS,NaN,SNV,NaN,GLI1,12,57865247.0,12.0,L908L,A>G,c.2724A>G,NM_005269.3,51.04,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3879.0,2.77,NaN,NaN,Not Detected,0.000000,9.45,Low,Not Detected,87.36,3.5,NaN,12/14/23,1/7/21,3/15/24,CRC,Not Applicable,Not Applicable,240111_A02019_0030_AHMHTTDSX7,/ghsfa/projects/bioinformatics/starfish/flowcentral240111_A02019_0030_AHMHTTDSX7.a6632f4d-dc3f-44f0-9ae1-0a8b55716eb1.20240113072253
54,N4496000000,214565_0,A0931263,B00578733,214565,0,1,SUCCESS,NaN,SNV,NaN,RECQL4,8,145739846.0,10.0,R562W,G>A,c.1684C>T,NM_004260.4,54.23,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV52881129,rs61759892,NaN,NaN,NaN,NaN,2560.0,4.71,NaN,NaN,Detected,0.035111,11.35,Low,Not Detected,18.72,2.0,NaN,12/26/23,1/1/22,3/15/24,CRC,Not Applicable,Not Applicable,240111_A02019_0030_AHMHTTDSX7,/ghsfa/projects/bioinformatics/starfish/flowcentral240111_A02019_0030_AHMHTTDSX7.a6632f4d-dc3f-44f0-9ae1-0a8b55716eb1.20240113072253


## Generate S3 MB caller outputs for this SOW (CRC and Lung)
###### Git clone Sirius-1.6.0_Release_Branch for region_file and control_file files for different S3 MB callers
###### Branch https://github.com/guardant/bip-products/blob/Sirius-1.6.0_Release_Branch/Sirius_LDT_Soft/Sirius_LDT_Soft.yaml#L653-L680
###### Confluence page to follow: https://guardanthealth.atlassian.net/wiki/spaces/LUN/pages/3554411001/Model+files+for+BIP+update
###### This is from Mingyang's notes: https://guardanthealth.atlassian.net/wiki/spaces/PHARMA/pages/3769729300/Sirius-1.5+vs+Sirius-1.6+and+module+rerun#How-to-run-the-MB-and-TFv2_gamma-callers:~:text=How%20to%20run%20the%20MB%20and%20TFv2_gamma%20callers

In [8]:
# Instead of using epicaller.methylation.methyl_caller funciton, bip-epigenomics module (branch 2.5 with Sirius 1.6 MB caller region/control/module files) is prefered.

def run_bip_epigenomics_module_wrapper_slurm(sample_meta_df, workdir):
    module_dir = '/ghsfa/projects/omni_v2/users/chlee/bip4/bip-epigenomics/' # branch 2.5
    # parameter_set_yaml = "/ghsfa/bik8s/test/bip-portal/outputs/Sirius_LDT_Soft_240905_020053/products/sirius/Sirius_LDT_Soft/Sirius_LDT_Soft.yaml" # branch Sirius 1.6
    parameter_set_yaml = "/ghsfa/projects/pharma/customers/OHSU/Infinity/OHSU01_IST01_Infinity/OHSU01_IST01_Infinity_01/data/update_MB_TF/bip-products/Sirius_LDT_Soft/Sirius_LDT_Soft.yaml" # branch Sirius 1.6


    jobs_dir = f"{workdir}/MB_module_SOW172_jobs"
    output_dir = f"{workdir}/MB_module_SOW172_output"

    # Check and create directories if not exists
    if not os.path.exists(jobs_dir):
        os.makedirs(jobs_dir)

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for _, row in sample_meta_df.iterrows():
        run_sample_id = row["GHSampleID"]
        fc_dir = row["fc_dir"]

        # # Check if there is a 'sirius' folder under the fc_dir. This is for Sirius 1.5 update
        # if os.path.exists(os.path.join(fc_dir, "sirius")):
        #     fc_dir = os.path.join(fc_dir, "sirius")

        mbd_hyper_molecules = f"{fc_dir}/{run_sample_id}/{run_sample_id}.mbd_hyper_molecules.tsv"
        bip_config_json_file = f"{fc_dir}/bip_config.json"
        
        if not os.path.exists(mbd_hyper_molecules):
            continue

        # Make a folder for each run_sample_id
        output_sample_dir = f"{output_dir}/{run_sample_id}"

        if not os.path.exists(output_sample_dir):
            os.makedirs(output_sample_dir)

        # Copy the bip_config.json for each sample from BIP output. This is essential for module running
        # (samples in the same batch shared the same bip_config.json)
        os.system(f"cp {bip_config_json_file} {output_sample_dir}")

        job_file = f"{jobs_dir}/MB_call_{run_sample_id}.job"
        shebang_content = (f"#!/bin/bash\n"
                           f"#SBATCH --job-name=MB_call_{run_sample_id}\n"
                           f"#SBATCH --nodes=1\n"
                           f"#SBATCH --ntasks=1\n"
                           f"#SBATCH --cpus-per-task=1\n"
                           f"#SBATCH --mem=60gb\n"
                           f"#SBATCH --partition=dev\n"
                           f"#SBATCH --time=01:00:00\n"
                           f"#SBATCH --output={jobs_dir}/MB_call_{run_sample_id}.log\n\n")
        
        with open(job_file, "w") as job_output:
            job_output.write(shebang_content)
            job_output.write(f"{module_dir}/module_entry msre_caller " + 
                             f"--in-hyper-molecules {mbd_hyper_molecules} " + 
                             f"--sample-id {run_sample_id} " + 
                             f"--output-dir {output_sample_dir}/ " +
                             f"--parameter-set {parameter_set_yaml} " + 
                             f"--threads 16 " + 
                             f"--memory-log {output_sample_dir}/{run_sample_id}-epigenomics_msre_caller.memory.log " +
                             f"--log-file {output_sample_dir}/{run_sample_id}--epigenomics_msre_caller.log")
            # print(job_output)    
        os.system(f"sbatch {job_file}")


In [9]:
#run the function to generate all the MB S3 caller ouputs for this sow  
run_bip_epigenomics_module_wrapper_slurm(df_sow172_2, 
                      f"{work_dir}")

Submitted batch job 96869681
Submitted batch job 96869682
Submitted batch job 96869683
Submitted batch job 96869684
Submitted batch job 96869685
Submitted batch job 96869686
Submitted batch job 96869687
Submitted batch job 96869688
Submitted batch job 96869689
Submitted batch job 96869690
Submitted batch job 96869691
Submitted batch job 96869692
Submitted batch job 96869693
Submitted batch job 96869694
Submitted batch job 96869695
Submitted batch job 96869696
Submitted batch job 96869697
Submitted batch job 96869698
Submitted batch job 96869699
Submitted batch job 96869700
Submitted batch job 96869701
Submitted batch job 96869702
Submitted batch job 96869703
Submitted batch job 96869704
Submitted batch job 96869705
Submitted batch job 96869706
Submitted batch job 96869707
Submitted batch job 96869708
Submitted batch job 96869709
Submitted batch job 96869710
Submitted batch job 96869711
Submitted batch job 96869712
Submitted batch job 96869713
Submitted batch job 96869714
Submitted batc

In [ ]:
# # Retrieve MB S3 CRC caller score and MB S3 Lung caller score based on the cancer type

# df_sow172_3 = df_sow172_2.copy()

# df_sow172_3["MB_score"] = np.nan

# mb_caller_result_folder = f"{work_dir}/MB_module_SOW172_output"
# problematic_rows = []  # List to store indices of problematic rows

# for index, crc_row in df_sow172_3.iterrows():
#     run_sample_id = crc_row["GHSampleID"]

#     # This is a mixed cancer type cohort
#     cancer_type = crc_row["Cancertype"]
#     if cancer_type == "CRC":
#         msre_mr_file = f"{mb_caller_result_folder}/{run_sample_id}/{run_sample_id}/{run_sample_id}.msre_crc_call.hdr.tsv"
#     else:
#         msre_mr_file = f"{mb_caller_result_folder}/{run_sample_id}/{run_sample_id}/{run_sample_id}.msre_lung_call.hdr.tsv"
#     try:
#         msre_df = pd.read_table(msre_mr_file)

#         if not msre_df.empty:
#             df_sow172_3.at[index, "MB_score"] = msre_df.loc[0, "methyl_score"]

#         else:
#             # Add the index to the list if msre_df is empty
#             problematic_rows.append(index) 
            
#     except Exception as e:
#         # Add the index to the list if there is any other exception
#         problematic_rows.append(index)  
#         print(f"Error processing file {msre_mr_file}: {str(e)}")

# # Create a DataFrame with the indices of the problematic rows
# df_problematic = df_sow172_3.loc[problematic_rows]

# print("Total sample size of SOW172 Infinity cumulative batches with NO MB output is " + str(df_problematic.shape[0]) + "\n") 

# # Returning the DataFrame with problematic rows
# df_sow172_3.head()


Total sample size of SOW172 Infinity cumulative batches with NO MB output is 0



,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Visit_name,Alt-NoAlt,Sample_status,Sample_comment,Variant_type,Indel_type,Gene,Chromosome,Position,Exon,Mut_aa,Mut_nt,Mut_cdna,Transcript,Percentage,Splice_effect,Somatic_status,Molecular_consequence,Fusion_chrom_b,Fusion_gene_b,Fusion_position_a,Fusion_position_b,Direction_a,Direction_b,Downstream_gene,Copy_number,CNV_type,COSMIC,dbSNP,ClinVar,ClinVarID,Functional_impact,Mutant_allele_status,Mol_count,Max_percentage,Alleletype,HRD_score,Tumor_methylation_status,Tumor_methylation_score,TMB_score,TMB_category,MSI_High,cfDNA_ng,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,Cancertype,Practice_name,Physician_name,runid,fc_dir,MB_score
3,N4496000000,151684_0,A0931212,B00578712,151684,0,0,FAIL,Possible Contamination,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.53,2.5,NaN,12/26/23,1/1/22,3/15/24,CRC,Not Applicable,Not Applicable,240206_A02048_0117_AHNNYCDSX7,/ghsfa/ivd/flowcentral/240206_A02048_0117_AHNNYCDSX7.1da433ba-2782-431f-a56a-b7257055cb67.20240207222712,9.862761
1,N4496000000,130869837_0,A0931211,B00578713,130869837,0,1,SUCCESS,NaN,SNV,NaN,NOVA1,14,27066580.0,1.0,D21D,G>A,c.63C>T,NM_002515.3,52.41,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs145050329,NaN,NaN,NaN,NaN,806.0,1.71,NaN,NaN,Detected,0.000671,10.63,Low,Not Detected,26.10,3.5,NaN,12/14/23,12/14/22,3/15/24,CRC,Not Applicable,Not Applicable,240206_A02048_0117_AHNNYCDSX7,/ghsfa/ivd/flowcentral/240206_A02048_0117_AHNNYCDSX7.1da433ba-2782-431f-a56a-b7257055cb67.20240207222712,-16.551620
52,N4496000000,23986_1,A0931261,B00578731,23986,1,1,SUCCESS,NaN,SNV,NaN,FZD7,2,202899471.0,1.0,P34L,C>T,c.101C>T,NM_003507.2,51.68,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs772362388,NaN,NaN,NaN,NaN,1292.0,0.39,NaN,NaN,Not Detected,0.000000,NaN,Not evaluable,Not Detected,183.04,4.0,NaN,12/14/23,1/26/23,3/15/24,LUNG,Not Applicable,Not Applicable,240111_A02019_0030_AHMHTTDSX7,/ghsfa/ivd/flowcentral/240111_A02019_0030_AHMHTTDSX7,-21.253684
51,N4496000000,200058744_0,A0931260,B00578732,200058744,0,1,SUCCESS,NaN,SNV,NaN,GLI1,12,57865247.0,12.0,L908L,A>G,c.2724A>G,NM_005269.3,51.04,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3879.0,2.77,NaN,NaN,Not Detected,0.000000,9.45,Low,Not Detected,87.36,3.5,NaN,12/14/23,1/7/21,3/15/24,CRC,Not Applicable,Not Applicable,240111_A02019_0030_AHMHTTDSX7,/ghsfa/ivd/flowcentral/240111_A02019_0030_AHMHTTDSX7,-20.211387
54,N4496000000,214565_0,A0931263,B00578733,214565,0,1,SUCCESS,NaN,SNV,NaN,RECQL4,8,145739846.0,10.0,R562W,G>A,c.1684C>T,NM_004260.4,54.23,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV52881129,rs61759892,NaN,NaN,NaN,NaN,2560.0,4.71,NaN,NaN,Detected,0.035111,11.35,Low,Not Detected,18.72,2.0,NaN,12/26/23,1/1/22,3/15/24,CRC,Not Applicable,Not Applicable,240111_A02019_0030_AHMHTTDSX7,/ghsfa/ivd/flowcentral/240111_A02019_0030_AHMHTTDSX7,15.905744


## Generate S3 MB CRC caller outputs (Sirius 1.5.0)
###### Git clone Sirius-1.5.0_Release_Branch for region_file and control_file files for different S3 MB callers
###### Branch https://github.com/guardant/bip-products/blob/Sirius-1.5.0-RLS/Sirius_LDT_Soft/Sirius_LDT_Soft.yaml#L610-L623
###### Cloned Sirius-1.5.0_Release_Branch dir: /home/hazhang/projects/Repo/bip-products/

In [6]:
# Make sure update the epicaller to 2.0.3 or above version
import epicaller
print(epicaller.__version__)

2.0.3


In [7]:
import epicaller
import epicaller.methylation.methyl_caller

def mb_crc_caller_sirius150_wrapper_slurm(sample_meta_df, workdir):
    region_file='/home/hazhang/projects/Repo/bip-products/models/Sirius/mafband_crc.classification_space.tsv'
    control_region_file='/home/hazhang/projects/Repo/bip-products/models/Sirius/mafband_crc.positive_control_regions.tsv'
    mr_model_file='/home/hazhang/projects/Repo/bip-products/models/Sirius/mafband_crc.threshold_set_to_zero.tsv'

    jobs_dir = f"{workdir}/MB_CRC_log"
    output_dir = f"{workdir}/MB_CRC_output"

    # Check and create directories if not exists
    if not os.path.exists(jobs_dir):
        os.makedirs(jobs_dir)

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for _, row in sample_meta_df.iterrows():
        run_sample_id = row["GHSampleID"]
        fc_dir = row["fc_dir"]
        flowcell_id = row["runid"]
        mbd_hyper_molecules = f"{fc_dir}/{run_sample_id}/{run_sample_id}.mbd_hyper_molecules.tsv"
        
        # Check if the molecule file exists
        if not os.path.exists(mbd_hyper_molecules):
            continue

        job_file = f"{jobs_dir}/MB_call_{run_sample_id}.job"
        shebang_content = (f"#!/bin/bash\n"
                           f"#SBATCH --job-name=MB_call_{run_sample_id}\n"
                           f"#SBATCH --nodes=1\n"
                           f"#SBATCH --ntasks=1\n"
                           f"#SBATCH --cpus-per-task=1\n"
                           f"#SBATCH --mem=30gb\n"
                           f"#SBATCH --partition=dev\n"
                           f"#SBATCH --time=00:10:00\n"
                           f"#SBATCH --output={jobs_dir}/MB_call_{run_sample_id}.log\n\n")

        with open(job_file, "w") as job_output:
            job_output.write(shebang_content)
            job_output.write("python3 -m epicaller.methylation.methyl_caller " + 
                             f"--hyper-molecule-file {mbd_hyper_molecules} " + 
                             f"--region-file {region_file} " +
                             f"--control-region-file {control_region_file} " + 
                             f"--model-file {mr_model_file} " +
                             "--model-name crc --min-frag-len 120 --max-frag-len 240 --min-cg-count 12 --max-cg-count 30 --cutsite-count-cutoff 1 --min-mapq 60 " + 
                             f"--sample-id {run_sample_id} --flowcell-id {flowcell_id} --outdir {output_dir}/")
            # print(job_output)    
        os.system(f"sbatch {job_file}")

In [8]:
# Generate the MB S3 CRC caller outputs (Sirius 1.5.0)
mb_crc_caller_sirius150_wrapper_slurm(df_sow172_2, f"{work_dir}")

Submitted batch job 97780752
Submitted batch job 97780753
Submitted batch job 97780754
Submitted batch job 97780755
Submitted batch job 97780756
Submitted batch job 97780757
Submitted batch job 97780758
Submitted batch job 97780759
Submitted batch job 97780760
Submitted batch job 97780761
Submitted batch job 97780762
Submitted batch job 97780763
Submitted batch job 97780764
Submitted batch job 97780765
Submitted batch job 97780766
Submitted batch job 97780767
Submitted batch job 97780768
Submitted batch job 97780769
Submitted batch job 97780770
Submitted batch job 97780771
Submitted batch job 97780772
Submitted batch job 97780773
Submitted batch job 97780774
Submitted batch job 97780775
Submitted batch job 97780776
Submitted batch job 97780777
Submitted batch job 97780778
Submitted batch job 97780779
Submitted batch job 97780780
Submitted batch job 97780781
Submitted batch job 97780782
Submitted batch job 97780783
Submitted batch job 97780784
Submitted batch job 97780785
Submitted batc

In [46]:
# Retrieve MB S3 CRC caller score and MB S3 Lung caller score based on the cancer type
df_sow172_3 = df_sow172_2.copy()

df_sow172_3["MB_score"] = np.nan

mb_caller_result_folder = f"{work_dir}/MB_module_SOW172_output"
crc_caller_result_folder = f"{work_dir}/MB_CRC_output"

problematic_rows = []  # List to store indices of problematic rows

for index, crc_row in df_sow172_3.iterrows():
    run_sample_id = crc_row["GHSampleID"]

    # This is a mixed cancer type cohort
    cancer_type = crc_row["Cancertype"]
    if cancer_type == "CRC":
        msre_mr_file = f"{crc_caller_result_folder}/{run_sample_id}.msre_crc_call.hdr.tsv"
    else:
        msre_mr_file = f"{mb_caller_result_folder}/{run_sample_id}/{run_sample_id}/{run_sample_id}.msre_lung_call.hdr.tsv"
    try:
        msre_df = pd.read_table(msre_mr_file)

        if not msre_df.empty:
            df_sow172_3.at[index, "MB_score"] = msre_df.loc[0, "methyl_score"] 

        else:
            # Add the index to the list if msre_df is empty
            problematic_rows.append(index) 
            
    except Exception as e:
        # Add the index to the list if there is any other exception
        problematic_rows.append(index)  
        print(f"Error processing file {msre_mr_file}: {str(e)}")

# Create a DataFrame with the indices of the problematic rows
df_problematic = df_sow172_3.loc[problematic_rows]

print("Total sample size of SOW172 Infinity cumulative batches with NO MB output is " + str(df_problematic.shape[0]) + "\n") 

# Returning the DataFrame with problematic rows
df_sow172_3.head()


Total sample size of SOW172 Infinity cumulative batches with NO MB output is 0



,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Visit_name,Alt-NoAlt,Sample_status,Sample_comment,Variant_type,Indel_type,Gene,Chromosome,Position,Exon,Mut_aa,Mut_nt,Mut_cdna,Transcript,Percentage,Splice_effect,Somatic_status,Molecular_consequence,Fusion_chrom_b,Fusion_gene_b,Fusion_position_a,Fusion_position_b,Direction_a,Direction_b,Downstream_gene,Copy_number,CNV_type,COSMIC,dbSNP,ClinVar,ClinVarID,Functional_impact,Mutant_allele_status,Mol_count,Max_percentage,Alleletype,HRD_score,Tumor_methylation_status,Tumor_methylation_score,TMB_score,TMB_category,MSI_High,cfDNA_ng,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,Cancertype,Practice_name,Physician_name,runid,fc_dir,MB_score
3,N4496000000,151684_0,A0931212,B00578712,151684,0,0,FAIL,Possible Contamination,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.53,2.5,NaN,12/26/23,1/1/22,3/15/24,CRC,Not Applicable,Not Applicable,240206_A02048_0117_AHNNYCDSX7,/ghsfa/ivd/flowcentral/240206_A02048_0117_AHNNYCDSX7.1da433ba-2782-431f-a56a-b7257055cb67.20240207222712,29.694824
1,N4496000000,130869837_0,A0931211,B00578713,130869837,0,1,SUCCESS,NaN,SNV,NaN,NOVA1,14,27066580.0,1.0,D21D,G>A,c.63C>T,NM_002515.3,52.41,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs145050329,NaN,NaN,NaN,NaN,806.0,1.71,NaN,NaN,Detected,0.000671,10.63,Low,Not Detected,26.10,3.5,NaN,12/14/23,12/14/22,3/15/24,CRC,Not Applicable,Not Applicable,240206_A02048_0117_AHNNYCDSX7,/ghsfa/ivd/flowcentral/240206_A02048_0117_AHNNYCDSX7.1da433ba-2782-431f-a56a-b7257055cb67.20240207222712,3.385752
52,N4496000000,23986_1,A0931261,B00578731,23986,1,1,SUCCESS,NaN,SNV,NaN,FZD7,2,202899471.0,1.0,P34L,C>T,c.101C>T,NM_003507.2,51.68,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs772362388,NaN,NaN,NaN,NaN,1292.0,0.39,NaN,NaN,Not Detected,0.000000,NaN,Not evaluable,Not Detected,183.04,4.0,NaN,12/14/23,1/26/23,3/15/24,LUNG,Not Applicable,Not Applicable,240111_A02019_0030_AHMHTTDSX7,/ghsfa/ivd/flowcentral/240111_A02019_0030_AHMHTTDSX7,-21.253684
51,N4496000000,200058744_0,A0931260,B00578732,200058744,0,1,SUCCESS,NaN,SNV,NaN,GLI1,12,57865247.0,12.0,L908L,A>G,c.2724A>G,NM_005269.3,51.04,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3879.0,2.77,NaN,NaN,Not Detected,0.000000,9.45,Low,Not Detected,87.36,3.5,NaN,12/14/23,1/7/21,3/15/24,CRC,Not Applicable,Not Applicable,240111_A02019_0030_AHMHTTDSX7,/ghsfa/ivd/flowcentral/240111_A02019_0030_AHMHTTDSX7,-0.341978
54,N4496000000,214565_0,A0931263,B00578733,214565,0,1,SUCCESS,NaN,SNV,NaN,RECQL4,8,145739846.0,10.0,R562W,G>A,c.1684C>T,NM_004260.4,54.23,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV52881129,rs61759892,NaN,NaN,NaN,NaN,2560.0,4.71,NaN,NaN,Detected,0.035111,11.35,Low,Not Detected,18.72,2.0,NaN,12/26/23,1/1/22,3/15/24,CRC,Not Applicable,Not Applicable,240111_A02019_0030_AHMHTTDSX7,/ghsfa/ivd/flowcentral/240111_A02019_0030_AHMHTTDSX7,35.789967


In [49]:
# To convert the mb crc score to original value. Cutoff was -20.2
df_sow172_crc = df_sow172_3.copy()

df_sow172_crc['MB_score'] = np.where(
    df_sow172_crc['Cancertype'] == 'CRC',  
    df_sow172_crc['MB_score'] - 20.2,  
    df_sow172_crc['MB_score'] 
)

df_sow172_crc.head()

,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Visit_name,Alt-NoAlt,Sample_status,Sample_comment,Variant_type,Indel_type,Gene,Chromosome,Position,Exon,Mut_aa,Mut_nt,Mut_cdna,Transcript,Percentage,Splice_effect,Somatic_status,Molecular_consequence,Fusion_chrom_b,Fusion_gene_b,Fusion_position_a,Fusion_position_b,Direction_a,Direction_b,Downstream_gene,Copy_number,CNV_type,COSMIC,dbSNP,ClinVar,ClinVarID,Functional_impact,Mutant_allele_status,Mol_count,Max_percentage,Alleletype,HRD_score,Tumor_methylation_status,Tumor_methylation_score,TMB_score,TMB_category,MSI_High,cfDNA_ng,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,Cancertype,Practice_name,Physician_name,runid,fc_dir,MB_score
3,N4496000000,151684_0,A0931212,B00578712,151684,0,0,FAIL,Possible Contamination,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.53,2.5,NaN,12/26/23,1/1/22,3/15/24,CRC,Not Applicable,Not Applicable,240206_A02048_0117_AHNNYCDSX7,/ghsfa/ivd/flowcentral/240206_A02048_0117_AHNNYCDSX7.1da433ba-2782-431f-a56a-b7257055cb67.20240207222712,9.494824
1,N4496000000,130869837_0,A0931211,B00578713,130869837,0,1,SUCCESS,NaN,SNV,NaN,NOVA1,14,27066580.0,1.0,D21D,G>A,c.63C>T,NM_002515.3,52.41,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs145050329,NaN,NaN,NaN,NaN,806.0,1.71,NaN,NaN,Detected,0.000671,10.63,Low,Not Detected,26.10,3.5,NaN,12/14/23,12/14/22,3/15/24,CRC,Not Applicable,Not Applicable,240206_A02048_0117_AHNNYCDSX7,/ghsfa/ivd/flowcentral/240206_A02048_0117_AHNNYCDSX7.1da433ba-2782-431f-a56a-b7257055cb67.20240207222712,-16.814248
52,N4496000000,23986_1,A0931261,B00578731,23986,1,1,SUCCESS,NaN,SNV,NaN,FZD7,2,202899471.0,1.0,P34L,C>T,c.101C>T,NM_003507.2,51.68,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs772362388,NaN,NaN,NaN,NaN,1292.0,0.39,NaN,NaN,Not Detected,0.000000,NaN,Not evaluable,Not Detected,183.04,4.0,NaN,12/14/23,1/26/23,3/15/24,LUNG,Not Applicable,Not Applicable,240111_A02019_0030_AHMHTTDSX7,/ghsfa/ivd/flowcentral/240111_A02019_0030_AHMHTTDSX7,-21.253684
51,N4496000000,200058744_0,A0931260,B00578732,200058744,0,1,SUCCESS,NaN,SNV,NaN,GLI1,12,57865247.0,12.0,L908L,A>G,c.2724A>G,NM_005269.3,51.04,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3879.0,2.77,NaN,NaN,Not Detected,0.000000,9.45,Low,Not Detected,87.36,3.5,NaN,12/14/23,1/7/21,3/15/24,CRC,Not Applicable,Not Applicable,240111_A02019_0030_AHMHTTDSX7,/ghsfa/ivd/flowcentral/240111_A02019_0030_AHMHTTDSX7,-20.541978
54,N4496000000,214565_0,A0931263,B00578733,214565,0,1,SUCCESS,NaN,SNV,NaN,RECQL4,8,145739846.0,10.0,R562W,G>A,c.1684C>T,NM_004260.4,54.23,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV52881129,rs61759892,NaN,NaN,NaN,NaN,2560.0,4.71,NaN,NaN,Detected,0.035111,11.35,Low,Not Detected,18.72,2.0,NaN,12/26/23,1/1/22,3/15/24,CRC,Not Applicable,Not Applicable,240111_A02019_0030_AHMHTTDSX7,/ghsfa/ivd/flowcentral/240111_A02019_0030_AHMHTTDSX7,15.589967


## Generate the epiTFv2 gamma score results

In [50]:
# Temporary solution for TFv2gamma score generation
df_sow172_3 = df_sow172_crc.copy()
df_sow172_3 = df_sow172_3[df_sow172_3['Sample_status'] != "FAIL"]
df_sow172_3.shape

# Save the data
df_sow172_3.to_csv(f"{work_dir}/df_sow172_3.csv", index = False)

In [51]:
# Step 1: save the input csv file
df_sow172_3 = pd.read_csv(f"{work_dir}/df_sow172_3.csv", header = 0)
df_sow172_3.head()

# Step 2 generate the slurm script
TFv2gamma_output_dir = f"{work_dir}/TFv2gamma_SOW172_output"

# Check and create directories if not exists
if not os.path.exists(TFv2gamma_output_dir):
    os.makedirs(TFv2gamma_output_dir)

In [15]:
# Step 2 generate the slurm script

# Define the contents of the slurm script
# Parts to updates
    # sample_list; out_path; slurm_script_path

slurm_script_content = f"""#! /bin/sh
#SBATCH --get-user-env
#SBATCH --job-name TFv2gamma_pancanTF
#SBATCH --partition dev
#SBATCH --mem-per-cpu 40G
#SBATCH --cpus-per-task 1
#SBATCH --ntasks-per-node 1
#SBATCH -o {TFv2gamma_output_dir}/epiTFv2gamma_%A.log
#SBATCH --array 1-1000%300

module load python/3.9.6;
export INSTALL_DIR=/home/mcai/workspace/tfv2_gamma/bip-pycommon;
source $INSTALL_DIR/.py3_venv/bin/activate;
export PYTHONPATH=$INSTALL_DIR:$PYTHONPATH;

sample_list={work_dir}/df_sow172_3.csv
out_path="{TFv2gamma_output_dir}"
mkdir -p $out_path

# Use Python to extract the necessary fields robustly
# Note that the parsing method is changed due to weired parsing issue of "fc_dir" of this sow
read sample_id fc_dir <<< $(python -c "import pandas as pd; import os
df = pd.read_csv('$sample_list')
# Adjusting SLURM_ARRAY_TASK_ID from 1-based to 0-based index
index = int(os.getenv('SLURM_ARRAY_TASK_ID', '1')) - 1
sample_id = df.iloc[index]['GHSampleID']
fc_dir = df.iloc[index]['fc_dir']
print(sample_id, fc_dir)")

# Prepend base path if fc_dir is relative
base_path="/ghsfa/ivd/flowcentral"
if [[ "$fc_dir" != /* ]]; then
    fc_dir="${{base_path}}/${{fc_dir}}"
fi

# Construct paths
sample_path="${{fc_dir}}/${{sample_id}}"
mbd_path="${{sample_path}}/${{sample_id}}.mbd_hyper_molecules.tsv"
methyl_qc_file="${{sample_path}}/${{sample_id}}.methyl_qc.tsv"
subpanel_qc_path="${{mbd_path/mbd_hyper_molecules/subpanel_qc_stats.hdr}}"

# Run the code
python -m epi_tumor_fraction.run_epi_tf $mbd_path $methyl_qc_file $out_path $sample_id /ghdevhome/home/mcai/workspace/tfv2_gamma/bip-products/models/Sirius/epi_tf_models/params.sirius.yaml --subpanel_qc_file $subpanel_qc_path
"""

# Save the slurm script to file
slurm_script_path = f"{work_dir}/run_TFv2gamma_sow172.sh"
with open(slurm_script_path, "w") as file:
    file.write(slurm_script_content)

In [16]:
# Step 3 run the script
os.system(f"sbatch {work_dir}/run_TFv2gamma_sow172.sh")

Submitted batch job 96879054


0

In [52]:
# Retrive the TFv2gamma value

# Remove the "FAIL" samples
df_sow172_4 = df_sow172_3[df_sow172_3["Sample_status"] != "FAIL"]

# Initialize a list to store rows where no TFv2gamma output was found
rows_no_tfv2 = []

df_sow172_4['pred_frac_force'] = None 

TFv2gamma_output_dir = f"{work_dir}/TFv2gamma_SOW172_output"

for index, row in df_sow172_4.iterrows():
    sample_id = row["GHSampleID"]
    file_path = f"{TFv2gamma_output_dir}/{sample_id}.epi_tf_model_predictions.tsv"

    file_path_tfv2 = glob.glob(file_path)
    if not file_path_tfv2:
        rows_no_tfv2.append(row)
    
    else:
        try:
            df_temp = pd.read_table(file_path)
            value_gamma = df_temp[df_temp['metric'] == 'epi_tf']['value'].values[0]
            df_sow172_4.at[index, 'pred_frac_force'] = value_gamma
            
        except Exception as e:
            print(f"An error occurred with {sample_id}: {str(e)}")

# Create a DataFrame with rows where folders were not found in any sources
df_no_tfv2 = pd.DataFrame(rows_no_tfv2)
print("Total sample size of SOW172 Infinity cumulative batches with NO TFv2gamma output is " + str(df_no_tfv2.shape[0]) + "\n") 


df_sow172_5 = df_sow172_4.copy()
df_sow172_5.head()

Total sample size of SOW172 Infinity cumulative batches with NO TFv2gamma output is 0



,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Visit_name,Alt-NoAlt,Sample_status,Sample_comment,Variant_type,Indel_type,Gene,Chromosome,Position,Exon,Mut_aa,Mut_nt,Mut_cdna,Transcript,Percentage,Splice_effect,Somatic_status,Molecular_consequence,Fusion_chrom_b,Fusion_gene_b,Fusion_position_a,Fusion_position_b,Direction_a,Direction_b,Downstream_gene,Copy_number,CNV_type,COSMIC,dbSNP,ClinVar,ClinVarID,Functional_impact,Mutant_allele_status,Mol_count,Max_percentage,Alleletype,HRD_score,Tumor_methylation_status,Tumor_methylation_score,TMB_score,TMB_category,MSI_High,cfDNA_ng,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,Cancertype,Practice_name,Physician_name,runid,fc_dir,MB_score,pred_frac_force
0,N4496000000,130869837_0,A0931211,B00578713,130869837,0,1,SUCCESS,NaN,SNV,NaN,NOVA1,14,27066580.0,1.0,D21D,G>A,c.63C>T,NM_002515.3,52.41,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs145050329,NaN,NaN,NaN,NaN,806.0,1.71,NaN,NaN,Detected,0.000671,10.63,Low,Not Detected,26.10,3.5,NaN,12/14/23,12/14/22,3/15/24,CRC,Not Applicable,Not Applicable,240206_A02048_0117_AHNNYCDSX7,/ghsfa/ivd/flowcentral/240206_A02048_0117_AHNNYCDSX7.1da433ba-2782-431f-a56a-b7257055cb67.20240207222712,-16.814248,0.000671
1,N4496000000,23986_1,A0931261,B00578731,23986,1,1,SUCCESS,NaN,SNV,NaN,FZD7,2,202899471.0,1.0,P34L,C>T,c.101C>T,NM_003507.2,51.68,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs772362388,NaN,NaN,NaN,NaN,1292.0,0.39,NaN,NaN,Not Detected,0.000000,NaN,Not evaluable,Not Detected,183.04,4.0,NaN,12/14/23,1/26/23,3/15/24,LUNG,Not Applicable,Not Applicable,240111_A02019_0030_AHMHTTDSX7,/ghsfa/ivd/flowcentral/240111_A02019_0030_AHMHTTDSX7,-21.253684,0.000665
2,N4496000000,200058744_0,A0931260,B00578732,200058744,0,1,SUCCESS,NaN,SNV,NaN,GLI1,12,57865247.0,12.0,L908L,A>G,c.2724A>G,NM_005269.3,51.04,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3879.0,2.77,NaN,NaN,Not Detected,0.000000,9.45,Low,Not Detected,87.36,3.5,NaN,12/14/23,1/7/21,3/15/24,CRC,Not Applicable,Not Applicable,240111_A02019_0030_AHMHTTDSX7,/ghsfa/ivd/flowcentral/240111_A02019_0030_AHMHTTDSX7,-20.541978,0.000862
3,N4496000000,214565_0,A0931263,B00578733,214565,0,1,SUCCESS,NaN,SNV,NaN,RECQL4,8,145739846.0,10.0,R562W,G>A,c.1684C>T,NM_004260.4,54.23,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV52881129,rs61759892,NaN,NaN,NaN,NaN,2560.0,4.71,NaN,NaN,Detected,0.035111,11.35,Low,Not Detected,18.72,2.0,NaN,12/26/23,1/1/22,3/15/24,CRC,Not Applicable,Not Applicable,240111_A02019_0030_AHMHTTDSX7,/ghsfa/ivd/flowcentral/240111_A02019_0030_AHMHTTDSX7,15.589967,0.035111
4,N4496000000,200052495_0,A0931262,B00578734,200052495,0,1,SUCCESS,NaN,SNV,NaN,MYOD1,11,17741814.0,1.0,A162V,C>T,c.485C>T,NM_002478.5,51.44,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs150053079,NaN,NaN,NaN,NaN,1829.0,1.95,NaN,NaN,Not Detected,0.000000,NaN,Not evaluable,Not Detected,38.01,3.5,NaN,12/12/23,10/6/22,3/15/24,CRC,Not Applicable,Not Applicable,240111_A02019_0030_AHMHTTDSX7,/ghsfa/ivd/flowcentral/240111_A02019_0030_AHMHTTDSX7,-21.087247,0.000918


### For checking troubleshooting purposes on TFv2gamma (no need to run)


In [ ]:
# This is for checking troubleshooting purposes

# Sample that has no TFv2gamma output "B00481373"
df_sow164_4 = df_sow187_3[df_sow187_3["GHSampleID"] == "B00625530"]
# df_sow157_4 = df_sow157_3[df_sow157_3["GHSampleID"] == "B00558789"]

# Save the data for test sh run
df_sow164_4.to_csv(f"{work_dir}/df_sow164_4.csv", index = False)
df_sow164_4

,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Visit_name,Alt-NoAlt,Sample_status,Sample_comment,Variant_type,Indel_type,Gene,Chromosome,Position,Exon,Mut_aa,Mut_nt,Mut_cdna,Transcript,Percentage,Splice_effect,Somatic_status,Molecular_consequence,Fusion_chrom_b,Fusion_gene_b,Fusion_position_a,Fusion_position_b,Direction_a,Direction_b,Downstream_gene,Copy_number,CNV_type,COSMIC,dbSNP,ClinVar,ClinVarID,Functional_impact,Mutant_allele_status,Mol_count,Max_percentage,Alleletype,HRD_score,Tumor_methylation_status,Tumor_methylation_score,TMB_score,TMB_category,MSI_High,cfDNA_ng,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,Cancertype,Practice_name,Physician_name,Tumor_stage,ctDNA_detected,cfDNA_ng_yield,Unnamed: 0,runid,fc_dir,Tumor_methylation_status_mapped,MB_score
2,D926UC00001,6802077988,B00625530,B00625530,E4306001,SCREEN,1.0,SUCCESS,NaN,SNV,NaN,SOX2,3,181430551.0,1.0,G135S,G>A,c.403G>A,NM_003106.4,50.86,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2767.0,0.14,NaN,NaN,NaN,NaN,NaN,Not evaluable,Not Detected,75.92,1.0,NaN,01/10/2024,NaN,07/17/2024,NaN,Not Applicable,Not Applicable,NaN,NaN,NaN,NaN,240619_A01744_0224_AHG3Y7DSXC,/ghsfa/ivd/flowcentral/240619_A01744_0224_AHG3Y7DSXC.5cae61cd-3024-4c67-8bcc-d1e799ee5671.20240626013927,NaN,-22.813004


In [ ]:
print(df_sow164_4['fc_dir'])


2    /ghsfa/ivd/flowcentral/240619_A01744_0224_AHG3Y7DSXC.5cae61cd-3024-4c67-8bcc-d1e799ee5671.20240626013927
Name: fc_dir, dtype: object


In [ ]:
# STEP 3: run the test script with debugging prints
# os.system(f"sbatch {work_dir}/run_TFv2gamma_pancanTF_test187.sh")
os.system(f"sbatch {work_dir}/run_TFv2gamma_pancanTF_test187_new_1.sh")

Submitted batch job 87999489


0

## Generate the updated report of the SOW based on "TFv2" rule 

In [53]:
def update_tumor_methylation_status_MBcutoff_withTFv2filter(df, MB_score_column, MB18_cutoff, MB_cutoff):
    """
    Update the 'Tumor_methylation_status' and 'Tumor_methylation_score' columns in the DataFrame based on provided cutoff values ('TFv2' rule).
    
    Parameters:
        df (DataFrame): The DataFrame to update. Must with two columns "MB_score_column" and TFv2gamma ("pred_frac_force")
        MB_score_column (str): The column name for MB caller scores.
        MB18_cutoff (float): The cutoff value for the MB18 (must match MB caller type!).
        MB_cutoff (float): The cutoff value for the MB90/MB95/MB97 (must match MB caller type!).

    Notes:
        cutoff refenence: https://docs.google.com/presentation/d/1U6TYnddThQHxo9-Ahxpty2ie2_SpnIEcICbTobH4s4g/edit#slide=id.g2f5b7ef9d14_0_50
                          https://guardanthealth.atlassian.net/wiki/spaces/LUN/pages/3554411001/Model+files+for+BIP+update
        Elmira: Lung model 97%： -20.417943 Lung model 95%： -20.665406 
    """

    # case1: Addressing flipped calls where "Tumor_methylation_status" is "Not Detected" but TFv2 rule is triggered.
            # Then flip it to "Detected" and replace value "Tumor_methylation_score" with "pred_frac_force"
    conditions_case1 = (
        (df['Tumor_methylation_status'] == 'Not Detected') &
        ((df[MB_score_column] >= MB_cutoff) |
         ((df[MB_score_column] >= MB18_cutoff) & (df[MB_score_column] < MB_cutoff) & (df['pred_frac_force'] > 0.0005)))
    )

    # case2: Addressing flipped calls where "Tumor_methylation_status" is "Detected" but TFv2 rule is NOT triggered. 
            # Then flip it to "Not Detected" and replace value "Tumor_methylation_score" with 0
    conditions_case2 = (
        (df['Tumor_methylation_status'] == 'Detected') &
        ~((df[MB_score_column] >= MB_cutoff) |
          ((df[MB_score_column] >= MB18_cutoff) & (df[MB_score_column] < MB_cutoff) & (df['pred_frac_force'] > 0.0005)))
    )

    # case3: "Tumor_methylation_status" is nan for FAIL samples. No updates

    # Apply updates based on conditions
    df.loc[conditions_case1, 'Tumor_methylation_status'] = 'Detected'
    df.loc[conditions_case1, 'Tumor_methylation_score'] = df.loc[conditions_case1, 'pred_frac_force']

    df.loc[conditions_case2, 'Tumor_methylation_status'] = 'Not Detected'
    df.loc[conditions_case2, 'Tumor_methylation_score'] = 0

    # Replace the udpated "Tumor_methylation_score" with "pred_frac_force" when "Tumor_methylation_status" is "Detected"
    df['Tumor_methylation_score'] = np.where(df['Tumor_methylation_status'] == "Detected", df['pred_frac_force'], df['Tumor_methylation_score'])

    return df

In [54]:
def update_tumor_methylation_status_MB(df, MB_score_column, MB_cutoff):
    """
    Update the 'Tumor_methylation_status' and 'Tumor_methylation_score' columns in the DataFrame based on provided cutoff values ('TFv2' rule).
    
    Parameters:
        df (DataFrame): The DataFrame to update. Must with two columns "MB_score_column" and TFv2gamma ("pred_frac_force")
        MB_score_column (str): The column name for MB caller scores.
        MB_cutoff (float): The cutoff value for the MB90/MB95/MB97 (must match MB caller type!).

    Notes:
        cutoff refenence: https://docs.google.com/presentation/d/1U6TYnddThQHxo9-Ahxpty2ie2_SpnIEcICbTobH4s4g/edit#slide=id.g2f5b7ef9d14_0_50
                          https://guardanthealth.atlassian.net/wiki/spaces/LUN/pages/3554411001/Model+files+for+BIP+update
        Elmira: Lung model 97%： -20.417943 Lung model 95%： -20.665406 
    """

    # case1: Addressing flipped calls where "Tumor_methylation_status" is "Not Detected" but TFv2 rule is triggered.
            # Then flip it to "Detected" and replace value "Tumor_methylation_score" with "pred_frac_force"
    conditions_case1 = (
        (df['Tumor_methylation_status'] == 'Not Detected') & (df[MB_score_column] >= MB_cutoff)
    )

    # case2: Addressing flipped calls where "Tumor_methylation_status" is "Detected" but TFv2 rule is NOT triggered. 
            # Then flip it to "Not Detected" and replace value "Tumor_methylation_score" with 0
    conditions_case2 = (
        (df['Tumor_methylation_status'] == 'Detected') & ~(df[MB_score_column] >= MB_cutoff)
    )

    # case3: "Tumor_methylation_status" is nan for FAIL samples. No updates

    # Apply updates based on conditions
    df.loc[conditions_case1, 'Tumor_methylation_status'] = 'Detected'
    df.loc[conditions_case1, 'Tumor_methylation_score'] = df.loc[conditions_case1, 'pred_frac_force']

    df.loc[conditions_case2, 'Tumor_methylation_status'] = 'Not Detected'
    df.loc[conditions_case2, 'Tumor_methylation_score'] = 0

    # Replace the udpated "Tumor_methylation_score" with "pred_frac_force" when "Tumor_methylation_status" is "Detected"
    df['Tumor_methylation_score'] = np.where(df['Tumor_methylation_status'] == "Detected", df['pred_frac_force'], df['Tumor_methylation_score'])

    return df

In [55]:
# Some cutoffs are shared by Elmira 
# MB Lung amd CRC caller cutoffs for the SOW
MB18_lung_cutoff = -22.421763
MB90_lung_cutoff = -20.941267
MB95_lung_cutoff = -20.665406
MB97_lung_cutoff = -20.417943
MB98_lung_cutoff = -20.343954721795292

MB18_crc_cutoff = -22.4522
MB90_crc_cutoff = -21.249985
MB95_crc_cutoff = -20.774269
MB97_crc_cutoff = -20.454949
MB98_crc_cutoff = -20.2

In [56]:
# Update for different cutoffs with TFv2 filter

# Straitify by cancer type
df_172_lung = df_sow172_5[df_sow172_5['Cancertype'] == "LUNG"]
df_172_crc = df_sow172_5[df_sow172_5['Cancertype'] == "CRC"]

# For MB90 with TFv2 rescue
# Lung samples
df_sow172_updated_90_withTFv2filter_lung = update_tumor_methylation_status_MBcutoff_withTFv2filter(df_172_lung, 'MB_score', MB18_lung_cutoff, MB90_lung_cutoff)
# CRC samples
df_sow172_updated_90_withTFv2filter_crc = update_tumor_methylation_status_MBcutoff_withTFv2filter(df_172_crc, 'MB_score', MB18_crc_cutoff, MB90_crc_cutoff)
# Concatenate the dataframes
df_sow172_updated = pd.concat([df_sow172_updated_90_withTFv2filter_lung, df_sow172_updated_90_withTFv2filter_crc], ignore_index=True)


# # For MB95 with TFv2 rescue
# # Lung samples
# df_sow172_updated_95_withTFv2filter_lung = update_tumor_methylation_status_MBcutoff_withTFv2filter(df_172_lung, 'MB_score', MB18_lung_cutoff, MB95_lung_cutoff)
# # CRC samples
# df_sow172_updated_95_withTFv2filter_crc = update_tumor_methylation_status_MBcutoff_withTFv2filter(df_172_crc, 'MB_score', MB18_crc_cutoff, MB95_crc_cutoff)
# # Concatenate the dataframes
# df_sow172_updated = pd.concat([df_sow172_updated_95_withTFv2filter_lung, df_sow172_updated_95_withTFv2filter_crc], ignore_index=True)


# # For MB97 with TFv2 rescue
# # Lung samples
# df_sow172_updated_97_withTFv2filter_lung = update_tumor_methylation_status_MBcutoff_withTFv2filter(df_172_lung, 'MB_score', MB18_lung_cutoff, MB97_lung_cutoff)
# # CRC samples
# df_sow172_updated_97_withTFv2filter_crc = update_tumor_methylation_status_MBcutoff_withTFv2filter(df_172_crc, 'MB_score', MB18_crc_cutoff, MB97_crc_cutoff)
# # Concatenate the dataframes
# df_sow172_updated = pd.concat([df_sow172_updated_97_withTFv2filter_lung, df_sow172_updated_97_withTFv2filter_crc], ignore_index=True)

df_sow172_updated.tail()

,Study_ID,Customer_SampleID,GHRequestID,GHSampleID,Patient_ID,Visit_name,Alt-NoAlt,Sample_status,Sample_comment,Variant_type,Indel_type,Gene,Chromosome,Position,Exon,Mut_aa,Mut_nt,Mut_cdna,Transcript,Percentage,Splice_effect,Somatic_status,Molecular_consequence,Fusion_chrom_b,Fusion_gene_b,Fusion_position_a,Fusion_position_b,Direction_a,Direction_b,Downstream_gene,Copy_number,CNV_type,COSMIC,dbSNP,ClinVar,ClinVarID,Functional_impact,Mutant_allele_status,Mol_count,Max_percentage,Alleletype,HRD_score,Tumor_methylation_status,Tumor_methylation_score,TMB_score,TMB_category,MSI_High,cfDNA_ng,Plasma_ml_input,Plasma_ml_remaining,Received_date,Bloodcoll_date,Reported_date,Cancertype,Practice_name,Physician_name,runid,fc_dir,MB_score,pred_frac_force
323,N4496000000,200007565_0,A0931402,B00579342,200007565,0,1,SUCCESS,NaN,SNV,NaN,MSH3,5,80074538.0,17.0,NaN,G>A,c.2319-1G>A,NM_002439.5,52.69,splice_acceptor_variant,germline,splice_acceptor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs866260675,NaN,NaN,NaN,NaN,440.0,1.08,NaN,NaN,Detected,0.000876,13.00,Low,Not Detected,26.78,2.5,NaN,12/14/23,5/30/22,3/15/24,CRC,Not Applicable,Not Applicable,240113_A01810_0295_BHMJK7DSX7,/ghsfa/ivd/flowcentral/240113_A01810_0295_BHMJK7DSX7.de53cf23-14a0-4f78-8858-42a6051599f3.20240115115751,-18.316711,0.000876
324,N4496000000,153610_0,A0931407,B00579346,153610,0,1,SUCCESS,NaN,SNV,NaN,KMT2C,7,151856098.0,44.0,T3840T,T>A,c.11520A>T,NM_170606.3,51.99,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rs747821479,NaN,NaN,NaN,NaN,1465.0,0.39,NaN,NaN,Detected,0.000525,NaN,Not evaluable,Not Detected,15.86,3.0,NaN,12/26/23,1/1/22,3/15/24,CRC,Not Applicable,Not Applicable,240113_A01810_0295_BHMJK7DSX7,/ghsfa/ivd/flowcentral/240113_A01810_0295_BHMJK7DSX7.de53cf23-14a0-4f78-8858-42a6051599f3.20240115115751,-18.617986,0.000525
325,N4496000000,214650_0,A0931411,B00579360,214650,0,1,SUCCESS,NaN,SNV,NaN,SOCS3,17,76354964.0,2.0,R71R,G>A,c.213C>T,NM_003955.5,53.07,NaN,germline,synonymous,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV100434553,rs528779263,NaN,NaN,NaN,NaN,1561.0,1.13,NaN,NaN,Detected,0.01422,2.84,Low,Not Detected,40.98,2.5,NaN,12/26/23,1/1/22,3/15/24,CRC,Not Applicable,Not Applicable,240113_A01810_0295_BHMJK7DSX7,/ghsfa/ivd/flowcentral/240113_A01810_0295_BHMJK7DSX7.de53cf23-14a0-4f78-8858-42a6051599f3.20240115115751,3.268953,0.01422
326,N4496000000,200007441_0,A0931378,B00579361,200007441,0,1,SUCCESS,NaN,SNV,NaN,FZD8,10,35929671.0,1.0,E229D,C>A,c.687G>T,NM_031866.3,58.06,NaN,germline,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,225.0,0.10,NaN,NaN,Detected,0.002393,NaN,Not evaluable,Not Detected,39.31,2.0,NaN,12/14/23,5/10/22,3/15/24,CRC,Not Applicable,Not Applicable,240113_A01810_0294_AHMMTGDSX7,/ghsfa/ivd/flowcentral/240113_A01810_0294_AHMMTGDSX7.838c525c-3c6c-427e-a21b-e75de9c10a9e.20240115115421,-21.160084,0.002393
327,N4496000000,130862923_0,A0931309,B00580361,130862923,0,1,SUCCESS,Possible Contamination Warning,SNV,NaN,RBM10,X,47041361.0,16.0,V569I,G>A,c.1705G>A,NM_005676.5,80.83,NaN,somatic,missense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,COSV61311461,rs782440714,NaN,NaN,NaN,NaN,3652.0,80.83,NaN,NaN,Detected,0.868343,NaN,Not evaluable,Not Detected,105.56,3.5,NaN,12/14/23,10/28/22,3/15/24,CRC,Not Applicable,Not Applicable,240114_A02019_0033_BHMHNYDSX7,/ghsfa/ivd/flowcentral/240114_A02019_0033_BHMHNYDSX7.f4f1c92b-92f4-4115-9b59-353bc9967c78.20240116000137,34.262197,0.868343


In [69]:
# Update for different cutoffs WITHOUT TFv2 filter

# # For MB95 WITHOUT TFv2 rescue
# # Lung samples
# df_sow172_updated_95_lung = update_tumor_methylation_status_MB(df_172_lung, 'MB_score', MB95_lung_cutoff)
# # CRC samples
# df_sow172_updated_95_crc = update_tumor_methylation_status_MB(df_172_crc, 'MB_score', MB95_crc_cutoff)
# # Concatenate the dataframes
# df_sow172_updated = pd.concat([df_sow172_updated_95_lung, df_sow172_updated_95_crc], ignore_index=True)


# # For MB97 WITHOUT TFv2 rescue
# # Lung samples
# df_sow172_updated_97_lung = update_tumor_methylation_status_MB(df_172_lung, 'MB_score', MB97_lung_cutoff)
# # CRC samples
# df_sow172_updated_97_crc = update_tumor_methylation_status_MB(df_172_crc, 'MB_score', MB97_crc_cutoff)
# # Concatenate the dataframes
# df_sow172_updated = pd.concat([df_sow172_updated_97_lung, df_sow172_updated_97_crc], ignore_index=True)


# For MB98 WITHOUT TFv2 rescue
# Lung samples
df_sow172_updated_98_lung = update_tumor_methylation_status_MB(df_172_lung, 'MB_score', MB98_lung_cutoff)
# CRC samples
df_sow172_updated_98_crc = update_tumor_methylation_status_MB(df_172_crc, 'MB_score', MB98_crc_cutoff)
# Concatenate the dataframes
df_sow172_updated = pd.concat([df_sow172_updated_98_lung, df_sow172_updated_98_crc], ignore_index=True)

# df_sow172_updated.head()

In [70]:
# Genearte the final report

# Subset the updated report for updated MB score and TFv2gamma columns
df_sow172_5_sub = df_sow172_updated[['GHSampleID', 'Tumor_methylation_status', 'Tumor_methylation_score']]
df_sow172_5_sub_dedup = df_sow172_5_sub.drop_duplicates(subset = 'GHSampleID')
print("Total sample size of SOW172 Infinity cumulative batches with updated MB score and TFv2gamma is " + str(df_sow172_5_sub_dedup.shape[0]) + "\n") 

# Drop duplicates of original report
df_sow172_all_dedup = df_sow172_all.drop_duplicates(subset = "GHSampleID")
print("Total sample size of original SOW187 Infinity cumulative batches is " + str(df_sow172_all_dedup.shape[0]) + "\n") 

# Check failed samples
df_sow172_all_dedup.Sample_status.value_counts()


Total sample size of SOW172 Infinity cumulative batches with updated MB score and TFv2gamma is 328

Total sample size of original SOW187 Infinity cumulative batches is 340



SUCCESS    328
FAIL        12
Name: Sample_status, dtype: int64

In [59]:
def suppress_methylation_score_higher_than_1(df, score_column='Tumor_methylation_score', id_column='GHSampleID'):
    # Identify rows where the score is greater than 1
    rows_to_modify = df[df[score_column] > 1].copy()

    # Remove duplicates based on GHSampleID and Tumor_methylation_score
    unique_rows_to_modify = rows_to_modify[[id_column, score_column]].drop_duplicates()

    # Print the unique rows that will be modified with their original scores and sample IDs
    if not unique_rows_to_modify.empty:
        print("Unique rows with scores > 1 before modification:")
        print(unique_rows_to_modify)

    # Modify the dataframe by capping the scores at 0.99
    df[score_column] = np.where(df[score_column] > 1, 0.99, df[score_column])

    # Show the rows after modification, ensuring they are also unique
    modified_rows = df.loc[rows_to_modify.index, [id_column, score_column]].drop_duplicates()
    if not modified_rows.empty:
        print("\nUnique rows after modification:")
        print(modified_rows)

    return df

In [71]:
# For Infinity report

# Merge the updated values into df_sow172_final based on 'GHSampleID'
df_merged = df_sow172_all.merge(df_sow172_5_sub_dedup[['GHSampleID', 'Tumor_methylation_status', 'Tumor_methylation_score']],
                                  on='GHSampleID', 
                                  how='left', 
                                  suffixes=('', '_updated'))

# Update the original columns with the values from the merged columns if they are not NaN
df_merged['Tumor_methylation_status'] = df_merged['Tumor_methylation_status_updated'].combine_first(df_merged['Tumor_methylation_status'])
df_merged['Tumor_methylation_score'] = df_merged['Tumor_methylation_score_updated'].combine_first(df_merged['Tumor_methylation_score'])


# Drop the temporary updated columns
df_merged.drop(columns=['Tumor_methylation_status_updated', 'Tumor_methylation_score_updated'], inplace=True)

# Ensure the dataframe maintains the original order and columns of df_sow??_final
df_sow172_infinity_final_all = df_merged[df_sow172_all.columns]

# Add TFv2 suppression rule. Apply secondary rule to corrct TFv2 score > 1 to 0.99
# Ad-hoc process
df_sow172_infinity_final_all_new = suppress_methylation_score_higher_than_1(df_sow172_infinity_final_all)

# Save the data
# df_sow172_infinity_final_all_new.to_csv(f"{work_dir}/20250108_AstraZeneca_SOW172_340Samples_TFv2_InfinityReport_MB97_withTFv2filter.csv", index = False)
# df_sow172_infinity_final_all_new.to_csv(f"{work_dir}/20250108_AstraZeneca_SOW172_340Samples_TFv2_InfinityReport_MB98.csv", index = False)

# df_sow172_infinity_final_all_new.to_csv(f"{work_dir}/20250122_AstraZeneca_SOW172_340Samples_TFv2_InfinityReport_MB90_withTFv2filter.csv", index = False)
df_sow172_infinity_final_all_new.to_csv(f"{work_dir}/20250122_AstraZeneca_SOW172_340Samples_TFv2_InfinityReport_MB98.csv", index = False)
# df_sow172_infinity_final_all_new.to_csv(f"{work_dir}/20250122_AstraZeneca_SOW172_340Samples_TFv2_InfinityReport_MB95.csv", index = False)
# df_sow172_infinity_final_all_new.to_csv(f"{work_dir}/20250122_AstraZeneca_SOW172_340Samples_TFv2_InfinityReport_MB97.csv", index = False)



Unique rows with scores > 1 before modification:
     GHSampleID Tumor_methylation_score
9919  B00579297                1.034266

Unique rows after modification:
     GHSampleID Tumor_methylation_score
9919  B00579297                    0.99


In [58]:
# Check the detection

# Define file paths
file_paths = [
    "/ghsfa/projects/pharma/projects/sirius_pharma/hazhang_projects/AZ_TFv2_update_SOW172_01082025/20250108_AstraZeneca_SOW172_340Samples_TFv2_InfinityReport_MB90_withTFv2filter.csv",
    "/ghsfa/projects/pharma/projects/sirius_pharma/hazhang_projects/AZ_TFv2_update_SOW172_01082025/20250108_AstraZeneca_SOW172_340Samples_TFv2_InfinityReport_MB95_withTFv2filter.csv",
    "/ghsfa/projects/pharma/projects/sirius_pharma/hazhang_projects/AZ_TFv2_update_SOW172_01082025/20250108_AstraZeneca_SOW172_340Samples_TFv2_InfinityReport_MB97_withTFv2filter.csv",
    "/ghsfa/projects/pharma/projects/sirius_pharma/hazhang_projects/AZ_TFv2_update_SOW172_01082025/20250108_AstraZeneca_SOW172_340Samples_TFv2_InfinityReport_MB95.csv",
    "/ghsfa/projects/pharma/projects/sirius_pharma/hazhang_projects/AZ_TFv2_update_SOW172_01082025/20250108_AstraZeneca_SOW172_340Samples_TFv2_InfinityReport_MB97.csv",
    "/ghsfa/projects/pharma/projects/sirius_pharma/hazhang_projects/AZ_TFv2_update_SOW172_01082025/20250108_AstraZeneca_SOW172_340Samples_TFv2_InfinityReport_MB98.csv",

]

# Initialize an empty list to store the counts
summary = []

# Loop through each file
for file_path in file_paths:
    # Read the CSV file
    df = pd.read_csv(file_path)
    # Remove duplicates
    df = df.drop_duplicates(subset = "GHSampleID")

    # Count the values in the "Tumor_methylation_status" column
    status_counts = df["Tumor_methylation_status"].value_counts()
    detected = status_counts.get("Detected", 0)
    not_detected = status_counts.get("Not Detected", 0)
    total = detected + not_detected
    
    # Calculate detection rate
    detection_rate = (detected / total) * 100 if total > 0 else 0
    
    # Extract specificity from the file name
    specificity = "_".join(file_path.split("/")[-1].split("_")[-2:])
    
    # Add the counts and new columns to the summary
    summary.append({
        "File": file_path.split("/")[-1],
        "Detected": detected,
        "Not Detected": not_detected,
        "Detection Rate %": round(detection_rate, 2),
        "Specificity": specificity
    })

# Convert the summary into a DataFrame for better visualization
summary_df = pd.DataFrame(summary)

# Display the result
summary_df


,File,Detected,Not Detected,Detection Rate %,Specificity
0,20250108_AstraZeneca_SOW172_340Samples_TFv2_InfinityReport_MB90_withTFv2filter.csv,233,95,71.04,MB90_withTFv2filter.csv
1,20250108_AstraZeneca_SOW172_340Samples_TFv2_InfinityReport_MB95_withTFv2filter.csv,212,116,64.63,MB95_withTFv2filter.csv
2,20250108_AstraZeneca_SOW172_340Samples_TFv2_InfinityReport_MB97_withTFv2filter.csv,208,120,63.41,MB97_withTFv2filter.csv
3,20250108_AstraZeneca_SOW172_340Samples_TFv2_InfinityReport_MB95.csv,191,137,58.23,InfinityReport_MB95.csv
4,20250108_AstraZeneca_SOW172_340Samples_TFv2_InfinityReport_MB97.csv,182,146,55.49,InfinityReport_MB97.csv
5,20250108_AstraZeneca_SOW172_340Samples_TFv2_InfinityReport_MB98.csv,176,152,53.66,InfinityReport_MB98.csv


In [72]:
# Check the detection

# Define file paths
file_paths = [
    "/ghsfa/projects/pharma/projects/sirius_pharma/hazhang_projects/AZ_TFv2_update_SOW172_01082025/20250122_AstraZeneca_SOW172_340Samples_TFv2_InfinityReport_MB90_withTFv2filter.csv",
    "/ghsfa/projects/pharma/projects/sirius_pharma/hazhang_projects/AZ_TFv2_update_SOW172_01082025/20250108_AstraZeneca_SOW172_340Samples_TFv2_InfinityReport_MB95_withTFv2filter.csv",
    "/ghsfa/projects/pharma/projects/sirius_pharma/hazhang_projects/AZ_TFv2_update_SOW172_01082025/20250108_AstraZeneca_SOW172_340Samples_TFv2_InfinityReport_MB97_withTFv2filter.csv",
    "/ghsfa/projects/pharma/projects/sirius_pharma/hazhang_projects/AZ_TFv2_update_SOW172_01082025/20250122_AstraZeneca_SOW172_340Samples_TFv2_InfinityReport_MB95.csv",
    "/ghsfa/projects/pharma/projects/sirius_pharma/hazhang_projects/AZ_TFv2_update_SOW172_01082025/20250122_AstraZeneca_SOW172_340Samples_TFv2_InfinityReport_MB97.csv",
    "/ghsfa/projects/pharma/projects/sirius_pharma/hazhang_projects/AZ_TFv2_update_SOW172_01082025/20250122_AstraZeneca_SOW172_340Samples_TFv2_InfinityReport_MB98.csv",

]

# Initialize an empty list to store the counts
summary = []

# Loop through each file
for file_path in file_paths:
    # Read the CSV file
    df = pd.read_csv(file_path)
    # Remove duplicates
    df = df.drop_duplicates(subset = "GHSampleID")

    # Count the values in the "Tumor_methylation_status" column
    status_counts = df["Tumor_methylation_status"].value_counts()
    detected = status_counts.get("Detected", 0)
    not_detected = status_counts.get("Not Detected", 0)
    total = detected + not_detected
    
    # Calculate detection rate
    detection_rate = (detected / total) * 100 if total > 0 else 0
    
    # Extract specificity from the file name
    specificity = "_".join(file_path.split("/")[-1].split("_")[-2:])
    
    # Add the counts and new columns to the summary
    summary.append({
        "File": file_path.split("/")[-1],
        "Detected": detected,
        "Not Detected": not_detected,
        "Detection Rate %": round(detection_rate, 2),
        "Specificity": specificity
    })

# Convert the summary into a DataFrame for better visualization
summary_df = pd.DataFrame(summary)

# Display the result
summary_df


,File,Detected,Not Detected,Detection Rate %,Specificity
0,20250122_AstraZeneca_SOW172_340Samples_TFv2_InfinityReport_MB90_withTFv2filter.csv,228,100,69.51,MB90_withTFv2filter.csv
1,20250108_AstraZeneca_SOW172_340Samples_TFv2_InfinityReport_MB95_withTFv2filter.csv,212,116,64.63,MB95_withTFv2filter.csv
2,20250108_AstraZeneca_SOW172_340Samples_TFv2_InfinityReport_MB97_withTFv2filter.csv,208,120,63.41,MB97_withTFv2filter.csv
3,20250122_AstraZeneca_SOW172_340Samples_TFv2_InfinityReport_MB95.csv,184,144,56.10,InfinityReport_MB95.csv
4,20250122_AstraZeneca_SOW172_340Samples_TFv2_InfinityReport_MB97.csv,178,150,54.27,InfinityReport_MB97.csv
5,20250122_AstraZeneca_SOW172_340Samples_TFv2_InfinityReport_MB98.csv,172,156,52.44,InfinityReport_MB98.csv
